# Gym Install


In [1]:
!bash pip install gym[box2d]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 374 kB 4.7 MB/s 
     |████████████████████████████████| 18.3 MB 137 kB/s 
     |████████████████████████████████| 1.8 MB 67.4 MB/s 
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Failed to build box2d-py
    Running setup.py install for box2d-py ... done
  DEPRECATION: box2d-py was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  7 19:54:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    55W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from collections import deque
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from copy import deepcopy
from collections import deque
from random import randrange
from torchvision import transforms
import random

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Utils


In [4]:
5

def from_tuple_to_tensor(tuple_of_np):
        tensor = np.zeros((len(tuple_of_np), tuple_of_np[0].shape[0], tuple_of_np[0].shape[1], tuple_of_np[0].shape[2]))
        for i, x in enumerate(tuple_of_np):
            tensor[i] = np.asarray(x)
        return tensor
        
def device(force_cpu=True):
    return "cuda" if torch.cuda.is_available() and not force_cpu else "cpu"                

# Buffer


In [15]:


class SumTree:
    def __init__(self, size):
        self.nodes = [0] * (2 * size - 1)
        self.data = [0] * size

        self.size = size
        self.count = 0
        self.real_size = 0

    @property
    def total(self):
        return self.nodes[0]

    def update(self, data_idx, value):
        idx = data_idx + self.size - 1  
        change = value - self.nodes[idx]

        self.nodes[idx] = value

        parent = (idx - 1) // 2
        while parent >= 0:
            self.nodes[parent] += change
            parent = (parent - 1) // 2

    def add(self, value, data):
        self.data[self.count] = data
        self.update(self.count, value)

        self.count = (self.count + 1) % self.size
        self.real_size = min(self.size, self.real_size + 1)

    def get(self, cumsum):
        assert cumsum <= self.total

        idx = 0
        while 2 * idx + 1 < len(self.nodes):
            left, right = 2*idx + 1, 2*idx + 2

            if cumsum <= self.nodes[left]:
                idx = left
            else:
                idx = right
                cumsum = cumsum - self.nodes[left]

        data_idx = idx - self.size + 1

        return data_idx, self.nodes[idx], self.data[data_idx]

    def __repr__(self):
        return f"SumTree(nodes={self.nodes.__repr__()}, data={self.data.__repr__()})"

class PrioritizedReplayBuffer:
    def __init__(self, env, buffer_size, eps=1e-2, alpha=0.1, beta=0.1):
        self.tree = SumTree(size=buffer_size)

        # PER params
        self.eps = eps  # minimal priority, prevents zero probabilities
        self.alpha = alpha  # determines how much prioritization is used, α = 0 corresponding to the uniform case
        self.beta = beta  # determines the amount of importance-sampling correction, b = 1 fully compensate for the non-uniform probabilities
        self.max_priority = eps  # priority for new samples, init as eps

        # transition: state, action, reward, next_state, done
        self.state = torch.empty(buffer_size, env.observation_space._shape[0],env.observation_space._shape[1],env.observation_space._shape[2], dtype=torch.float)
        self.action = torch.empty(buffer_size, dtype=torch.float)
        self.reward = torch.empty(buffer_size, dtype=torch.float)
        self.next_state = torch.empty(buffer_size, env.observation_space._shape[0],env.observation_space._shape[1],env.observation_space._shape[2], dtype=torch.float)
        self.done = torch.empty(buffer_size, dtype=torch.int)

        self.count = 0
        self.real_size = 0
        self.size = buffer_size

    def add(self, state, action, reward, done, next_state):

        # store transition index with maximum priority in sum tree
        self.tree.add(self.max_priority, self.count)

        # store transition in the buffer
        self.state[self.count] = torch.as_tensor(state)
        self.action[self.count] = torch.as_tensor(action)
        self.reward[self.count] = torch.as_tensor(reward)
        self.next_state[self.count] = torch.as_tensor(next_state)
        self.done[self.count] = torch.as_tensor(done)

        # update counters
        self.count = (self.count + 1) % self.size
        self.real_size = min(self.size, self.real_size + 1)

    def sample(self, batch_size):
        assert self.real_size >= batch_size, "buffer contains less samples than batch size"

        sample_idxs, tree_idxs = [], []
        priorities = torch.empty(batch_size, 1, dtype=torch.float)

        # To sample a minibatch of size k, the range [0, p_total] is divided equally into k ranges.
        # Next, a value is uniformly sampled from each range. Finally the transitions that correspond
        # to each of these sampled values are retrieved from the tree. (Appendix B.2.1, Proportional prioritization)
        segment = self.tree.total / batch_size
        for i in range(batch_size):
            a, b = segment * i, segment * (i + 1)

            cumsum = random.uniform(a, b)
            # sample_idx is a sample index in buffer, needed further to sample actual transitions
            # tree_idx is a index of a sample in the tree, needed further to update priorities
            tree_idx, priority, sample_idx = self.tree.get(cumsum)

            priorities[i] = priority
            tree_idxs.append(tree_idx)
            sample_idxs.append(sample_idx)

        probs = priorities / self.tree.total
        weights = (self.real_size * probs) ** -self.beta
        weights = weights / weights.max()

        batch = (
            self.state[sample_idxs],
            self.action[sample_idxs],
            self.reward[sample_idxs],
            self.done[sample_idxs],
            self.next_state[sample_idxs]
        )
        return batch, weights, tree_idxs

    def update_priorities(self, data_idxs, priorities):
        if isinstance(priorities, torch.Tensor):
            pass

        for data_idx, priority in zip(data_idxs, priorities):
            # The first variant we consider is the direct, proportional prioritization where p_i = |δ_i| + eps,
            # where eps is a small positive constant that prevents the edge-case of transitions not being
            # revisited once their error is zero. (Section 3.3)
            priority = (priority + self.eps) ** self.alpha

            self.tree.update(data_idx, priority)
            self.max_priority = max(self.max_priority, priority)        

# Q_NETWORK


In [6]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

#with warnings.catch_warnings():
#    warnings.filterwarnings("ignore", category=UserWarning)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Net(nn.Module):
    def __init__(self,n_inputs, n_outputs,hidden_size = 32,n_actions = 14, bias=True):
        super(Net, self).__init__()
        self.hidden_size = hidden_size
        self.n_frames = n_inputs
        self.conv1 = nn.Conv2d(self.n_frames, hidden_size, 7)
        self.conv2 = nn.Conv2d(hidden_size, hidden_size, 5)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, n_actions)
        self.device = device


    def forward(self, x):
        # Convolutions

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        # Global Max Pooling
        batch_size = x.shape[0]
        x = x.reshape(batch_size, self.hidden_size, -1).max(axis=2).values

        # Layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=1)

        return x
    

    def to(self, device):
        ret = super().to(device)
        ret.device = device
        return ret    


class Q_network(nn.Module):

    def __init__(self, env,  learning_rate=1e-4):
        super(Q_network, self).__init__()

        n_outputs = 14

        #self.network = Net( ?? , ??)
        self.network = Net( 1, n_outputs)

        self.optimizer = torch.optim.Adam(self.network.parameters(),
                                          lr=learning_rate)

    def greedy_action(self, state):
        # greedy action = ??
        # greedy_a = 0
        qvals = self.get_qvals(state)
        greedy_a = torch.max(qvals, dim=-1)[1].item()
        return greedy_a

    def get_qvals(self, state):
        #out = ???
        out = self.network.forward(state)
        return out 

# Policy


In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
class Policy(nn.Module):
    continuous = True

    def __init__(self, env = gym.make('CarRacing-v2', continuous = True, render_mode="rgb_array"), rew_thre = 400, learning_rate=0.1, initial_epsilon=0.9, batch_size= 64):

        super(Policy, self).__init__()
        self.env = env


        self.network = Q_network(env, learning_rate)
        self.target_network = deepcopy(self.network)
        self.buffer = PrioritizedReplayBuffer(env, 50000)
        self.epsilon = initial_epsilon
        self.batch_size = batch_size
        self.window = 50
        self.reward_threshold = rew_thre
        self.step_count = 0
        self.episode = 0
        self.rewards = 0
        self.update_loss = []
        self.training_rewards = []
        self.training_loss = []
        self.mean_training_rewards = []
        self.sync_eps = []

        self.gs = transforms.Grayscale()
        self.rs = transforms.Resize((64,64))

        self.moves = np.array([[1,1,0],[0.5,1,0],[0.1,1,0],
            [1,0.5,0],[0.5,0.5,0],[0.1,0.5,0],
            [-1,1,0],[-0.5,1,0],[-0.1,1,0],
            [-1,0.5,0],[-0.5,0.5,0],[-0.1,0.5,0],
            [0,1,0],[0,0.5,0]])

    def preproc_state(self, state):
        # State Preprocessing
        state = state[:83,:].transpose(2,0,1) #Torch wants images in format (channels, height, width)
        state = torch.from_numpy(state).float()
        state = self.gs(state) # grayscale
        state = self.rs(state) # resize
        return state/255 # normalize 
                
    
    def act(self, state):
        state = self.preproc_state(state).unsqueeze(0)
        return self.moves[self.network.greedy_action(state)]

    def take_step(self, mode='exploit'):
        # choose action with epsilon greedy
        if mode == 'explore':
                action = randrange(14)

        else:
                action = self.network.greedy_action(self.preproc_state(self.s_0).unsqueeze(0))


        #simulate action
        s_1, r, done, _= self.env.step(self.moves[action])

        #put experience in the buffer
    
        self.buffer.add(self.s_0, action, r, done, s_1)

        self.rewards += r

        self.s_0 = s_1.copy()

        self.step_count += 1
        if done:

            self.s_0 = self.env.reset()
        return done    

    def train(self,
     gamma=0.99, 
     max_episodes=500,
     network_update_frequency=10,
     network_sync_frequency=200):
        
        self.gamma = gamma

        self.loss_function = nn.MSELoss()
        self.s_0 = self.env.reset()

        # Populate replay buffer
        for i in range(1000):
            self.take_step(mode='explore')
        ep = 0
        training = True
        self.populate = False
        while training:

            self.s_0= self.env.reset()

            self.rewards = 0
            done = False
            while not done:

                if ((ep % 5) == 0):
                    self.env.render()

                p = np.random.random()
                if p < self.epsilon:
                    done = self.take_step(mode='explore')
                    #print("explore")
                else:
                    done = self.take_step(mode='exploit')
                    #print("train")
                # Update network
                if self.step_count % network_update_frequency == 0:
                    self.update()
                # Sync networks
                if self.step_count % network_sync_frequency == 0:
                    self.target_network.load_state_dict(
                        self.network.state_dict())
                    self.sync_eps.append(ep)

                if done:
                    if self.epsilon >= 0.05:
                        self.epsilon = self.epsilon * 0.95
                    ep += 1
                    if self.rewards > 2000:
                        self.training_rewards.append(2000)
                    elif self.rewards > 1000:
                        self.training_rewards.append(1000)
                    elif self.rewards > 500:
                        self.training_rewards.append(500)
                    else:
                        self.training_rewards.append(self.rewards)
                    if len(self.update_loss) == 0:
                        self.training_loss.append(0)
                    else:
                        self.training_loss.append(np.mean(self.update_loss))

                    mean_rewards = np.mean(self.training_rewards[-self.window:])
                    mean_loss = np.mean(self.training_loss[-self.window:])
                    self.mean_training_rewards.append(mean_rewards)
                    print(
                        "Episode {:d} Mean Rewards {:.2f}  Episode reward = {:.2f}   mean loss = {:.2f}\n".format(
                            ep, mean_rewards, self.rewards, mean_loss), end="")
                    
                    print("miao")

                    if ep >= max_episodes:
                        training = False
                        print('\nEpisode limit reached.')
                        break
                    if mean_rewards >= self.reward_threshold:
                        training = False
                        print('\nEnvironment solved in {} episodes!'.format(
                            ep))
                        #break
            self.save()            
        # save models
        self.save()

    def save(self):
        torch.save(self.state_dict(), 'model.pt')
        model_save_name = 'model.pt'
        path = F"/content/gdrive/My Drive/{model_save_name}" 
        torch.save(self.state_dict(), path)

    def load(self):
        self.load_state_dict(torch.load('model.pt'))

    def calculate_loss(self, batch, weights=None):
        #extract info from batch
        states, actions, rewards, dones, next_states = list(batch)

        #transform in torch tensors
        rewards = torch.FloatTensor(rewards).reshape(-1, 1)
        actions = torch.LongTensor(np.array(actions)).reshape(-1, 1)
        dones = torch.IntTensor(dones).reshape(-1, 1)
        states = from_tuple_to_tensor(states)
        next_states = from_tuple_to_tensor(next_states)

        states = torch.vstack([self.preproc_state(state).unsqueeze(0) for state in states])
        qvals = self.network.get_qvals(states)
        #states = torch.stack([self.preproc_state(state) for state in states]).unsequeeze(0)

        qvals = torch.gather(qvals, 1, actions)

        # target Q(s,a) = ??
        next_states = torch.vstack([self.preproc_state(state).unsqueeze(0) for state in next_states])
        next_qvals= self.network.get_qvals(next_states)
        next_qvals_max = torch.max(next_qvals, dim=-1)[0].reshape(-1, 1)
        target_qvals = rewards + (1 - dones)*self.gamma*next_qvals_max

        loss = self.loss_function(qvals, target_qvals)

        # loss = self.loss_function( Q(s,a) , target_Q(s,a))

        if weights is None:
            weights = torch.ones_like(qvals)

        td_error = torch.abs(qvals - target_qvals).detach()
        loss = torch.mean((qvals - target_qvals) ** 2 * weights)

        return loss, td_error

    def update(self):
        self.network.optimizer.zero_grad()
        batch, weights, tree_idxs = self.buffer.sample(batch_size=self.batch_size)
        loss, td_error = self.calculate_loss(batch, weights = weights)

        self.buffer.update_priorities(tree_idxs, td_error)



        loss.backward()
        self.network.optimizer.step()

        self.update_loss.append(loss.item())    
        

    



    

In [9]:
def train():
    agent = Policy()
    print(agent.network.network.device)
    agent.train()
    agent.save()

def loadtrain():
    agent = Policy()
    agent.load()
    agent.train()
    agent.save()       


In [ ]:
train()

cuda
Episode 1 Mean Rewards -95.93  Episode reward = -95.93   mean loss = 0.94
miao
Episode 2 Mean Rewards -87.91  Episode reward = -79.90   mean loss = 6.34
miao
Episode 3 Mean Rewards -86.27  Episode reward = -82.99   mean loss = 12.02
miao
Episode 4 Mean Rewards -84.13  Episode reward = -77.71   mean loss = 14.25
miao
Episode 5 Mean Rewards -83.33  Episode reward = -80.13   mean loss = 14.97
miao
Episode 6 Mean Rewards -83.11  Episode reward = -82.01   mean loss = 15.13
miao
Episode 7 Mean Rewards -83.13  Episode reward = -83.22   mean loss = 15.01
miao
Episode 8 Mean Rewards -82.27  Episode reward = -76.27   mean loss = 14.73
miao
Episode 9 Mean Rewards -82.26  Episode reward = -82.21   mean loss = 14.33
miao
Episode 10 Mean Rewards -81.78  Episode reward = -77.44   mean loss = 13.92
miao
Episode 11 Mean Rewards -81.83  Episode reward = -82.33   mean loss = 13.53
miao
Episode 12 Mean Rewards -81.91  Episode reward = -82.82   mean loss = 13.14
miao
Episode 13 Mean Rewards -82.05  Ep